In [1]:
import pandas as pd
import numpy as np
data_train = pd.read_excel("Data_Train.xlsx")
data_test = pd.read_excel('Data_Test.xlsx')

In [2]:
data_train.drop('New_Price',axis = 1,inplace = True)
data_test.drop('New_Price' , axis=1 , inplace = True)

In [3]:
y_train_data = data_train.iloc[:,-1].values
data_train = data_train.iloc[:,:-1]

In [4]:
data_train[data_train.isnull().any(axis=1)]
data_test[data_test.isna().any(axis=1)]

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats
23,Skoda Laura 1.8 TSI Ambition,Bangalore,2009,72000,Petrol,Manual,Second,17.5 kmpl,NaN,NaN,NaN
522,Toyota Etios Liva Diesel TRD Sportivo,Bangalore,2012,56600,Diesel,Manual,First,23.59 kmpl,NaN,NaN,NaN
525,Hyundai i20 new Sportz AT 1.4,Bangalore,2012,58000,Petrol,Automatic,Second,15.0 kmpl,NaN,NaN,NaN
614,Mahindra TUV 300 P4,Kolkata,2016,27000,Diesel,Manual,First,0.0 kmpl,NaN,NaN,NaN
624,BMW 5 Series 520d Sedan,Bangalore,2009,150000,Diesel,Automatic,Second,18.48 kmpl,NaN,NaN,NaN
632,Maruti Swift 1.3 VXi,Kolkata,2015,36009,Petrol,Manual,First,16.1 kmpl,NaN,NaN,NaN
658,Fiat Punto 1.4 Emotion,Jaipur,2010,65000,Petrol,Manual,Third,14.6 kmpl,NaN,NaN,NaN
666,Maruti Swift 1.3 VXi,Pune,2010,115000,Petrol,Manual,Second,16.1 kmpl,NaN,NaN,NaN
861,BMW 5 Series 520d Sedan,Chennai,2009,95000,Diesel,Automatic,Second,18.48 kmpl,NaN,NaN,NaN
883,Toyota Etios Liva V,Kochi,2012,59311,Petrol,Manual,First,18.3 kmpl,NaN,NaN,NaN


In [5]:
def change(series):
    shifted = []
    for i,item in enumerate(series):
        if (type(item)==float) or item.split()[0]=='null':
            shifted.append(np.nan)
        else:
            shifted.append(float(item.split()[0]))
    return shifted

In [6]:
str_col = ['Engine','Mileage','Power']
for item in str_col:
    data_train[item] = pd.Series(change(data_train[item]))
    data_test[item] = pd.Series(change(data_test[item]))

In [7]:
def brand_model_split(series):
    brand = []
    model = []
    for i,item in enumerate(series.Name):
        brand.append(item.split()[0])
        model.append(item[len(item.split()[0]):])
    series.insert(loc = 0,value = pd.Series(brand), column = 'Brand')
    series.insert(loc = 1,value = pd.Series(model), column = 'Model')
    series.drop('Name',axis=1,inplace=True)

In [8]:
brand_model_split(data_train)
brand_model_split(data_test)

In [9]:
from sklearn.preprocessing import LabelEncoder
enc_ind = [0,1,2,5,6,7]
le_enc = []
for j in enc_ind:
    le = LabelEncoder()
    le_enc.append(le)
    le.fit(pd.concat([data_train.iloc[:,j],data_test.iloc[:,j]],ignore_index = True))
    data_train.iloc[:,j] = le.transform(data_train.iloc[:,j])
    data_test.iloc[:,j] = le.transform(data_test.iloc[:,j])    

In [10]:
from sklearn.impute import SimpleImputer
imp_seats = SimpleImputer(missing_values = np.nan , strategy = 'constant' , fill_value = 4)
imp_power_engine_mileage = SimpleImputer(missing_values = np.nan , strategy = 'mean')
data_train['Seats'] = imp_seats.fit_transform(data_train['Seats'].values.reshape(-1,1))
data_train.iloc[:,8:11] = imp_power_engine_mileage.fit_transform(data_train.iloc[:,8:11])
data_test['Seats'] = imp_seats.transform(data_test['Seats'].values.reshape(-1,1))
data_test.iloc[:,8:11] = imp_power_engine_mileage.transform(data_test.iloc[:,8:11])

from sklearn.preprocessing import OneHotEncoder
enco_ind = ['Brand','Model','Location','Fuel_Type','Transmission','Owner_Type']
ohe_enc = []
for j in enco_ind:
    oe = OneHotEncoder()
    ohe_enc.append(oe)
    oe.fit([pd.concat([data_train[j],data_test[j]],ignore_index = True).values])
    data_train[j] = oe.transform([data_train[j]]).toarray()
    data_test[j] = oe.transform(data_test[j]).toarray()
    data_train = data_train[:,1:]
    data_test = data_test[:,1:]

In [11]:
data_train

,Brand,Model,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats
0,19,1797,9,2010,72000,0,1,0,26.60,998.0,58.16,5.0
1,11,498,10,2015,41000,1,1,0,19.67,1582.0,126.20,5.0
2,10,993,2,2011,46000,4,1,0,18.20,1199.0,88.70,5.0
3,19,676,2,2012,87000,1,1,0,20.77,1248.0,88.76,7.0
4,1,105,3,2013,40670,1,0,2,15.20,1968.0,140.80,5.0
5,11,591,5,2012,75000,3,1,0,21.10,814.0,55.20,5.0
6,23,1101,6,2013,86999,1,1,0,23.08,1461.0,63.10,5.0
7,30,973,9,2016,36000,1,0,0,11.36,2755.0,171.50,8.0
8,31,1699,10,2013,64430,1,1,0,20.54,1598.0,103.60,5.0
9,29,908,2,2012,65932,1,1,2,22.30,1248.0,74.00,5.0


In [12]:
from sklearn.preprocessing import StandardScaler
data_train = data_train[['Brand','Model','Location','Year','Fuel_Type','Transmission','Owner_Type','Kilometers_Driven','Mileage','Engine','Power','Seats']]
data_test = data_test[['Brand','Model','Location','Year','Fuel_Type','Transmission','Owner_Type','Kilometers_Driven','Mileage','Engine','Power','Seats']]
sc = StandardScaler()
sc.fit(data_train.iloc[:,7:11])
data_train.iloc[:,7:11] = sc.transform(data_train.iloc[:,7:11])
data_test.iloc[:,7:11] = sc.transform(data_test.iloc[:,7:11])

In [13]:
data_train

,Brand,Model,Location,Year,Fuel_Type,Transmission,Owner_Type,Kilometers_Driven,Mileage,Engine,Power,Seats
0,19,1797,9,2010,0,1,0,0.145315,1.847799,-1.039653,-1.035066,5.0
1,11,498,10,2015,1,1,0,-0.194369,0.335077,-0.065515,0.243242,5.0
2,10,993,2,2011,4,1,0,-0.139581,0.014197,-0.704376,-0.461293,5.0
3,19,676,2,2012,1,1,0,0.309678,0.575192,-0.622642,-0.460166,7.0
4,1,105,3,2013,1,0,2,-0.197985,-0.640661,0.578351,0.517541,5.0
5,11,591,5,2012,3,1,0,0.178188,0.647226,-1.346574,-1.090677,5.0
6,23,1101,6,2013,1,1,0,0.309667,1.079433,-0.267348,-0.942255,5.0
7,30,973,9,2016,1,0,0,-0.249157,-1.478879,1.891102,1.094320,8.0
8,31,1699,10,2013,1,1,0,0.062366,0.524986,-0.038826,-0.181358,5.0
9,29,908,2,2012,1,1,2,0.078824,0.909169,-0.622642,-0.737471,5.0


In [14]:
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV

import xgboost as xgb

In [15]:
from sklearn.model_selection import train_test_split
X_train , X_val , y_train  , y_val = train_test_split(data_train,y_train_data,test_size=0.2)

In [16]:
xgb_model = xgb.XGBRegressor(objective="reg:linear", random_state=0)
xgb_model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=True, subsample=1)

In [17]:
y_pred = xgb_model.predict(X_val)
mse=mean_squared_error(y_val, y_pred)
print(np.sqrt(mse))

3.646233477376638


In [18]:
y_pred2 = xgb_model.predict(data_test)
y_pred2 = pd.DataFrame(y_pred2 , columns = ['Price'])
#y_pred2.to_excel("output3.xlsx",index = False)

In [19]:
from sklearn.metrics import mean_squared_log_error
print(np.sqrt(mean_squared_log_error(y_val,y_pred)))

0.228719466167781


In [20]:
y_pred2.Price

0        3.993382
1        2.707158
2       18.021914
3        5.523703
4        4.031394
5       12.370261
6       22.573582
7        2.273616
8        3.450969
9       11.496263
10      17.831856
11       5.035713
12       3.407794
13       2.712869
14       4.874269
15      19.495893
16      27.316347
17       2.312258
18       8.273572
19       4.206578
20       5.254086
21       2.526246
22       3.978309
23       2.256117
24       5.012950
25       1.046540
26       0.776068
27       5.998921
28       8.548421
29       2.514218
          ...    
1204     6.314748
1205     4.827391
1206     5.305607
1207     3.927809
1208     1.950967
1209     4.053556
1210     2.788875
1211     4.031394
1212     9.778612
1213    17.278818
1214    20.795530
1215     4.981181
1216     7.927294
1217     6.321830
1218     4.611822
1219    12.405780
1220     6.066199
1221     8.121983
1222    18.955784
1223     4.137851
1224     5.775873
1225     2.062915
1226     4.251806
1227     7.247758
1228     3